In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Depression"
cohort = "GSE138297"

# Input paths
in_trait_dir = "../../input/GEO/Depression"
in_cohort_dir = "../../input/GEO/Depression/GSE138297"

# Output paths
out_data_file = "../../output/preprocess/Depression/GSE138297.csv"
out_gene_data_file = "../../output/preprocess/Depression/gene_data/GSE138297.csv"
out_clinical_data_file = "../../output/preprocess/Depression/clinical_data/GSE138297.csv"
json_path = "../../output/preprocess/Depression/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"The host response of IBS patients to allogenic and autologous faecal microbiota transfer"
!Series_summary	"In this randomised placebo-controlled trial, irritable bowel syndrome (IBS) patients were treated with faecal material from a healthy donor (n=8, allogenic FMT) or with their own faecal microbiota (n=8, autologous FMT). The faecal transplant was administered by whole colonoscopy into the caecum (30 g of stool in 150 ml sterile saline). Two weeks before the FMT (baseline) as well as two and eight weeks after the FMT, the participants underwent a sigmoidoscopy, and biopsies were collected at a standardised location (20-25 cm from the anal verge at the crossing with the arteria iliaca communis) from an uncleansed sigmoid. In patients treated with allogenic FMT, predominantly immune response-related genes sets were induced, with the strongest response two weeks after FMT. In patients treated with autologous FMT, predominantly metabolism-related g

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# Get the necessary libraries
import pandas as pd
import os
import json
from typing import Callable, Dict, Any, Optional

# 1. Gene Expression Data Availability
# Based on the background information, the dataset contains microarray analysis data
# Microarray analysis typically produces gene expression data, so this is likely gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# For trait (Depression), we need to see if there's a relevant field in the sample characteristics
# Looking at the data, there's no explicit "Depression" field, but we may infer it from "experimental condition"
# The study is about IBS (Irritable Bowel Syndrome) patients, not depression
trait_row = None  # Depression data is not available in this IBS study

# Age data is available at index 3
age_row = 3

# Gender data is available at index 1
gender_row = 1

# 2.2 Data Type Conversion
# For trait (Depression) - Not applicable as data is not available
def convert_trait(value):
    return None

# For age - Convert to continuous value
def convert_age(value):
    if value is None:
        return None
    try:
        # Extract the numeric part after the colon
        age_str = value.split(':')[1].strip()
        return float(age_str)
    except (ValueError, IndexError):
        return None

# For gender - Convert to binary (female=0, male=1)
def convert_gender(value):
    if value is None:
        return None
    try:
        # The format is already "sex (female=1, male=0): X"
        # But we'll convert to our standard of female=0, male=1
        gender_str = value.split(':')[1].strip()
        # Flip the values because in our standard female=0, male=1
        return 1 - int(gender_str)
    except (ValueError, IndexError):
        return None

# 3. Save Metadata
# Conduct initial filtering on the usability of the dataset
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Skip this step as trait_row is None, indicating clinical data for the trait is not available


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Depression/GSE138297/GSE138297_series_matrix.txt.gz


Gene data shape: (53617, 45)
First 20 gene/probe identifiers:
Index(['16650001', '16650003', '16650005', '16650007', '16650009', '16650011',
       '16650013', '16650015', '16650017', '16650019', '16650021', '16650023',
       '16650025', '16650027', '16650029', '16650031', '16650033', '16650035',
       '16650037', '16650041'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Based on the gene identifiers seen in the previous output, these appear to be probe identifiers 
# (possibly from an Illumina or Affymetrix microarray) rather than standard human gene symbols.
# The identifiers shown (like '16650001', '16650003', etc.) are numeric and don't match the 
# conventional format of human gene symbols (e.g., BRCA1, TP53).

# These identifiers will need to be mapped to standard gene symbols for meaningful analysis.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# Check if there are any platforms defined in the SOFT file that might contain annotation data
with gzip.open(soft_file, 'rt') as f:
    soft_content = f.read()

# Look for platform sections in the SOFT file
platform_sections = re.findall(r'^!Platform_title\s*=\s*(.+)$', soft_content, re.MULTILINE)
if platform_sections:
    print(f"Platform title found: {platform_sections[0]}")

# Try to extract more annotation data by reading directly from the SOFT file
# Look for lines that might contain gene symbol mappings
symbol_pattern = re.compile(r'ID_REF\s+Symbol|ID\s+Gene Symbol', re.IGNORECASE)
annotation_lines = []
with gzip.open(soft_file, 'rt') as f:
    for line in f:
        if symbol_pattern.search(line):
            annotation_lines.append(line)
            # Collect the next few lines to see the annotation structure
            for _ in range(10):
                annotation_lines.append(next(f, ''))

if annotation_lines:
    print("Found potential gene symbol mappings:")
    for line in annotation_lines:
        print(line.strip())

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("\nGene annotation preview:")
print(preview_df(gene_annotation, n=10))

# If we need an alternative source of mapping, check if there are any other annotation files in the cohort directory
cohort_files = os.listdir(in_cohort_dir)
annotation_files = [f for f in cohort_files if 'annotation' in f.lower() or 'platform' in f.lower()]
if annotation_files:
    print("\nAdditional annotation files found in the cohort directory:")
    for file in annotation_files:
        print(file)


Platform title found: [HuGene-2_1-st] Affymetrix Human Gene 2.1 ST Array [transcript (gene) version]



Gene annotation preview:
{'ID': ['16657436', '16657440', '16657445', '16657447', '16657450', '16657469', '16657473', '16657476', '16657480', '16657485'], 'probeset_id': ['16657436', '16657440', '16657445', '16657447', '16657450', '16657469', '16657473', '16657476', '16657480', '16657485'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1', 'chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'strand': ['+', '+', '+', '+', '+', '+', '+', '+', '+', '+'], 'start': ['12190', '29554', '69091', '160446', '317811', '329790', '367640', '459656', '523009', '557143'], 'stop': ['13639', '31109', '70008', '161525', '328581', '342507', '368634', '461954', '532878', '566063'], 'total_probes': [25.0, 28.0, 8.0, 13.0, 36.0, 27.0, 25.0, 27.0, 12.0, 13.0], 'gene_assignment': ['NR_046018 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102 /// NR_034090 // DDX11L9 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 9 // 15q26.3 // 100288486 /// NR_051985 // DDX11L9 // DEAD/H (

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Looking at the gene identifiers and the annotation data
# The gene expression data has identifiers like '16650001', '16650003', etc.
# In the gene annotation data, we see the 'ID' column matches this format

# 2. Extract the mapping between ID (probe) and gene symbols
# Looking at the gene_assignment column, we need to extract proper gene symbols from strings 
# like "NR_046018 // DDX11L1 // DEAD/H..."
prob_col = 'ID'
gene_col = 'gene_assignment'
mapping_df = get_gene_mapping(gene_annotation, prob_col, gene_col)

# Print the first few rows of the mapping dataframe to verify
print("Mapping dataframe preview (first 5 rows):")
print(mapping_df.head())

# 3. Apply the gene mapping to convert probe-level measurements to gene-level data
# This function will extract human gene symbols from the complex gene_assignment text,
# handle many-to-many relationships, and aggregate data by gene
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Normalize gene symbols to handle synonyms and variations
gene_data = normalize_gene_symbols_in_index(gene_data)

# Print information about the resulting gene expression dataframe
print(f"\nOriginal gene expression data shape (before mapping): {gene_data.shape}")
print(f"Number of unique genes after mapping: {len(gene_data.index.unique())}")

# Preview the first few genes and their expression values
print("\nFirst 5 genes and their expression values:")
print(gene_data.head())

# Save the gene data to a file for future use
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"\nGene expression data saved to: {out_gene_data_file}")


Mapping dataframe preview (first 5 rows):
         ID                                               Gene
0  16657436  NR_046018 // DDX11L1 // DEAD/H (Asp-Glu-Ala-As...
1  16657440  ENST00000473358 // MIR1302-11 // microRNA 1302...
2  16657445  NM_001005484 // OR4F5 // olfactory receptor, f...
3  16657447                                                ---
4  16657450  AK302511 // LOC100132062 // uncharacterized LO...



Original gene expression data shape (before mapping): (23274, 45)
Number of unique genes after mapping: 23274

First 5 genes and their expression values:
          GSM4104672  GSM4104673  GSM4104674  GSM4104675  GSM4104676  \
Gene                                                                   
A1BG        2.936132    2.803615    3.018072    2.549726    3.007617   
A1BG-AS1    1.384362    1.206311    1.430827    1.284927    1.401303   
A1CF        2.981835    2.945819    2.857161    2.967612    2.899801   
A2M         3.467480    3.621222    3.498609    3.411768    3.649466   
A2M-AS1     1.494063    1.592554    1.615310    1.529224    1.728269   

          GSM4104677  GSM4104678  GSM4104679  GSM4104680  GSM4104681  ...  \
Gene                                                                  ...   
A1BG        2.708221    2.533589    2.791591    3.039732    2.808330  ...   
A1BG-AS1    1.308060    1.106003    1.233850    1.264744    1.370906  ...   
A1CF        2.844060    2.943882


Gene expression data saved to: ../../output/preprocess/Depression/gene_data/GSE138297.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. We'll normalize gene symbols in the gene expression data
# Note: We've already done this in step 6, so we can skip this part

# 2. Link the clinical and genetic data
# Based on step 2, we confirmed that this dataset does not contain Depression data
# It's an IBS (Irritable Bowel Syndrome) study, not a Depression study
trait_row = None  # Depression data is not available in this IBS study

# Since trait data is not available, we can't proceed with clinical data extraction
print(f"No {trait} data available in this dataset.")

# Create a minimal dataframe with the trait column (though empty)
minimal_df = pd.DataFrame(columns=[trait])

# 5. Validate and save cohort information to record that this dataset is not usable for our purposes
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=False,  # No trait data available
    is_biased=False,           # Provide a boolean value instead of None
    df=minimal_df,             # Provide a minimal dataframe with trait column
    note="Dataset is about IBS patients, not Depression. The study was about faecal microbiota transfer in IBS patients."
)

# 6. Since the data is not usable for our Depression study, we don't save linked data
print("Dataset is not usable for Depression analysis. No linked data file saved.")

No Depression data available in this dataset.
Abnormality detected in the cohort: GSE138297. Preprocessing failed.
Dataset is not usable for Depression analysis. No linked data file saved.
